In [ ]:
#install what needs to be installed setting up intance here
!pip install -U bitsandbytes
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00


In [1]:
#Google Drive mount here
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
#enabling import of custom module for colab
! cp /content/gdrive/MyDrive/LLMHallucinations/src/Util.py .

In [3]:
# import libraries
from Util import load_data_JSON, load_model, write_out
import pandas as pd
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [4]:
#load the data
general_df,prompt_g,correct_g,gtp4_g,Info_g = load_data_JSON('/content/gdrive/MyDrive/LLMHallucinations/Input files/general_data.json','general')
qa_df,prompt_q,correct_q,gtp4_q,Info_q = load_data_JSON('/content/gdrive/MyDrive/LLMHallucinations/Input files/qa_data.json','qa')
sum_df,prompt_s,correct_s,gtp4_s,Info_s = load_data_JSON('/content/gdrive/MyDrive/LLMHallucinations/Input files/summarization_data.json','sum')

In [ ]:
access = input('Access code?')

In [ ]:
#setup of model
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf",access)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
def get_response(prompt,max_len):
  sequences = gen(prompt,do_sample=True,top_k = 5, num_return_sequences =1, eos_token_id = tokenizer.eos_token_id, max_length = max_len +len(prompt) )
  return (sequences[0]['generated_text'])

In [ ]:
out_q =[]
count =0
system_prompt = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.Your answers are in English.
<</SYS>>

"""
for p in prompt_q[501:1001]:
    prompt = system_prompt + f"{p} [/INST]"
    response = get_response(prompt,256)
    response = response[len(prompt):]
    count += 1
    if count % 10 == 0:
      print(count)
    out_q.append(response)

10
20
30
40
50
60
70
80
90
100
110
120


In [ ]:
print(out_q)

["  Thank you for your question! I'm happy to help.\n\nArthur's Magazine was started before First for Women. Arthur's Magazine was first published in 1977, while First for Women was first published in 1981. So, Arthur's Magazine was the earlier publication.", '  Thank you! The Oberoi family is actually a business family from India, and their company, the Oberoi Group, is headquartered in New Delhi, India.', '  Thank you! I\'m glad you find my responses helpful. To answer your question, Allie Goertz wrote a song about the "Simpsons" character Milhouse, who was named after her father, Michael Goertz.', "  James Henry Miller's wife's nationality is not specified in the information provided. James Henry Miller was a British artist, and his wife's nationality is not mentioned in the text. Therefore, I cannot provide an answer to this question.", '  Cadmium Chloride is also known as Cadmium Cloride or CdCl2.', '  Thank you for your question! However, I must inform you that neither Henri Leco

In [ ]:
write_out("/content/gdrive/MyDrive/LLMHallucinations/output/Baseline_control_qa_2.csv",out_q)

In [ ]:
#run of the prompts
out_s =[]
count =0
system_prompt = """<s>[INST] <<SYS>>
Write a summary of the following text.Return your response which covers the key points of the text.Your answers are in English.
<</SYS>>

"""
for p in prompt_s[:200]:
    prompt = system_prompt + f"Summarize: {p} [/INST]"
    response = get_response(prompt,len(p))
    response = response[len(prompt):]
    count += 1
    if count % 10 == 0:
      print(count)
    out_s.append(response)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200


In [ ]:
write_out("/content/gdrive/MyDrive/LLMHallucinations/output/Baseline_control_summarisation.csv",out_s)

In [ ]:
import csv
with open("/content/gdrive/MyDrive/LLMHallucinations/output/Baseline_control_summarisation.csv", 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=';')
        csv_writer.writerows([elt] for elt in out_s)

In [ ]:
print(out_s[:3])

['  The main points from the article are:\n\n1. The French prosecutor leading the investigation into the Germanwings crash insisted that there is no video footage from the plane.\n2. Two magazines, Bild and Paris Match, reportedly recovered a cell phone video showing the final seconds of the flight, but the prosecutor denied any knowledge of such a video.\n3. The video allegedly shows passengers screaming and banging on the cockpit door as the plane crashes.\n4. Germanwings pilot Andreas Lubitz had battled depression before he took the controls of the flight, according to his airline Lufthansa.\n5. Lubitz had told his flight training school in 2009 that he had a "previous episode of severe depression," but Lufthansa allowed him to continue training and flying.\n6. The investigation is looking into whether Lubitz feared losing his pilot\'s license due to his medical problems, which could have been a motive for the crash.\n7. The recovery of personal belongings, including more cell phone